<a href="https://colab.research.google.com/github/caiocezar400/DeepSea-Solutions/blob/main/Jacobiano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sympy as sp
import numpy as np
from sympy import symbols, cos, sin, pi, simplify, sympify, Matrix, eye, atan2, sqrt

def configurar_robo():
    """
    Configura o robô e retorna:
    - dh_params: Lista de parâmetros
    - joints: Lista de tipos ('R' ou 'P')
    - q_vars_list: Lista das variáveis simbólicas (q1, q2...)
    """
    config_str = input('\nConfiguração do robô (ex: RRR, RRP, RP): ').strip().upper()
    joints = [j for j in config_str if j in ['R', 'P']]
    N = len(joints)

    if N == 0:
        print("Erro: Nenhuma junta válida. Use 'R' ou 'P'.")
        return [], [], []

    print(f'\nRobô detectado com {N} juntas: {"-".join(joints)}')

    dh_params = []
    q_vars_list = [] # Lista para armazenar as variáveis de junta (q1, q2...)

    for i, junta_tipo in enumerate(joints):
        idx = i + 1
        q_var = symbols(f'q{idx}') # Cria a variável simbólica
        q_vars_list.append(q_var)  # Guarda na lista

        tipo_desc = "Revoluta (R)" if junta_tipo == 'R' else "Prismática (P)"
        print(f'\n--- Junta {idx}: {tipo_desc} ({q_var}) ---')

        a_str = input('a (cm): ').strip() or '0'
        alpha_str = input('alpha (rad): ').strip() or '0'
        a = sympify(a_str)
        alpha = sympify(alpha_str)

        if junta_tipo == 'R':
            d_str = input('d (cm): ').strip() or '0'
            theta_offset_str = input('theta offset (rad) [Enter para 0]: ').strip() or '0'
            d = sympify(d_str)
            theta_offset = sympify(theta_offset_str)
            theta = q_var + theta_offset
        elif junta_tipo == 'P':
            theta_str = input('theta (rad): ').strip() or '0'
            d_offset_str = input('d offset (cm) [Enter para 0]: ').strip() or '0'
            theta = sympify(theta_str)
            d_offset = sympify(d_offset_str)
            d = q_var + d_offset

        dh_params.append([a, alpha, d, theta])

    return dh_params, joints, q_vars_list

def forward_kinematics_symbolic(dh_params, N):
    """Calcula matrizes de transformação T01, T02... T0N."""
    def _tf_matrix(a, alpha, d, theta):
        c_th, s_th = sp.cos(theta), sp.sin(theta)
        c_al, s_al = sp.cos(alpha), sp.sin(alpha)
        return Matrix([
            [c_th, -s_th*c_al,  s_th*s_al, a*c_th],
            [s_th,  c_th*c_al, -c_th*s_al, a*s_th],
            [0,        s_al,          c_al,         d],
            [0,        0,                0,               1]
        ])

    T_matrices = []
    T_current = sp.eye(4)

    for i in range(N):
        p = dh_params[i]
        T_i = _tf_matrix(p[0], p[1], p[2], p[3])
        T_current = T_current @ T_i
        T_current = sp.simplify(T_current) # Simplificação passo a passo
        T_matrices.append(T_current)

    return T_matrices

def calculate_geometric_jacobian(joint_types, T_matrices, N):
    """
    Calcula Jg usando o método de propagação vetorial (Cross Product).
    """
    J_cols = []
    p_e = T_matrices[-1][0:3, 3] # Posição final

    p_0 = Matrix([0, 0, 0])
    z_0 = Matrix([0, 0, 1])

    for i in range(N):
        # Define p_(i-1) e z_(i-1)
        if i == 0:
            p_prev, z_prev = p_0, z_0
        else:
            p_prev = T_matrices[i-1][0:3, 3]
            z_prev = T_matrices[i-1][0:3, 2]

        if joint_types[i] == 'R':
            Jv = z_prev.cross(p_e - p_prev)
            Jw = z_prev
        else:
            Jv = z_prev
            Jw = Matrix([0, 0, 0])

        J_cols.append(Jv.col_join(Jw))

    Jg = Matrix.hstack(*J_cols)
    return sp.simplify(Jg)

def calculate_analytical_jacobian(T_final, q_vars):
    """
    Calcula Ja diferenciando a posição e os ângulos de Euler (ZYX).
    Ja = d(Pose)/dq
    """
    R = T_final[:3, :3]
    P = T_final[:3, 3]

    # Extração de ângulos de Euler ZYX (Yaw-Pitch-Roll)
    # R = Rz(alpha) * Ry(beta) * Rx(gamma)
    # alpha = Yaw, beta = Pitch, gamma = Roll

    # Sy calcula a magnitude para verificar singularidade (Gimbal Lock)
    sy = sp.sqrt(R[0,0]**2 + R[1,0]**2)

    # Fórmulas simbólicas para ZYX
    beta  = atan2(-R[2,0], sy)           # Pitch
    alpha = atan2(R[1,0], R[0,0])        # Yaw
    gamma = atan2(R[2,1], R[2,2])        # Roll

    # Vetor de Pose Operacional X = [x, y, z, yaw, pitch, roll]
    X_pose = Matrix([P[0], P[1], P[2], alpha, beta, gamma])

    # O Jacobiano Analítico é a derivada parcial de X em relação a q
    # O SymPy faz isso automaticamente com .jacobian()
    Ja = X_pose.jacobian(Matrix(q_vars))

    return sp.simplify(Ja)

def interacao_numerica(Jg_sym, Ja_sym, pos_sym, q_vars):
    """Realiza a substituição numérica e cálculos finais."""

    all_syms = Jg_sym.free_symbols.union(Ja_sym.free_symbols).union(pos_sym.free_symbols)

    if not all_syms:
        print("\n[!] O robô não possui variáveis livres.")
        return

    print("\n" + "="*40)
    print(" CÁLCULO NUMÉRICO ")
    print("="*40)
    print("Insira valores (ex: 0, pi/2, 10, 0.5):")

    subs_dict = {}
    # Ordena variáveis para entrada limpa
    sorted_syms = sorted(list(all_syms), key=lambda x: x.name)

    for s in sorted_syms:
        while True:
            val_str = input(f"{s} = ").strip() or '0'
            try:
                val = sp.sympify(val_str).evalf()
                subs_dict[s] = val
                break
            except:
                print("Valor inválido.")

    print("\nProcessando substituições (pode demorar um pouco)...")

    # Avaliação Numérica
    try:
        # Posição
        pos_vals = np.array(pos_sym.subs(subs_dict).evalf()).astype(float).flatten()

        # Jg
        Jg_vals = np.array(Jg_sym.subs(subs_dict).evalf()).astype(float)

        # Ja
        Ja_vals = np.array(Ja_sym.subs(subs_dict).evalf()).astype(float)

        np.set_printoptions(precision=4, suppress=True, linewidth=120)

        print(f"\nPosição do Efetuador (x, y, z):\n{pos_vals}")

        print("\nJacobiano Geométrico (Jg) [v; w]:")
        print(Jg_vals)

        print("\nJacobiano Analítico (Ja) [d_pos; d_euler_ZYX]:")
        print(Ja_vals)

        # Determinante (apenas se for quadrado)
        rows, cols = Jg_vals.shape
        if rows == cols:
            det_Jg = np.linalg.det(Jg_vals)
            print(f"\nDeterminante de Jg: {det_Jg:.4f}")
            if abs(det_Jg) < 1e-5:
                print("-> ALERTA: Configuração próxima de singularidade!")

        # Teste de Velocidade
        if input("\nCalcular velocidades? (s/n): ").lower() == 's':
            q_dot = []
            print(f"Insira velocidades para {cols} juntas:")
            for i in range(cols):
                v = float(input(f"q{i+1}_dot: ") or 0)
                q_dot.append(v)
            q_dot = np.array(q_dot)

            vel_geo = Jg_vals @ q_dot
            vel_ana = Ja_vals @ q_dot

            print("\nVelocidade Operacional (Geométrica - v, w):")
            print(f"Linear: {vel_geo[:3]}")
            print(f"Angular: {vel_geo[3:]}")

            print("\nTaxa de variação da Pose (Analítica - d_pos, d_euler):")
            print(f"Velocidade Linear: {vel_ana[:3]}")
            print(f"Taxa Euler (Yaw, Pitch, Roll): {vel_ana[3:]}")

    except Exception as e:
        print(f"Erro numérico: {e}")

def main():
    try:
        dh, joints, q_vars = configurar_robo()
        if not dh: return
        N = len(dh)

        print("\n1. Calculando Cinemática Direta...")
        T_mats = forward_kinematics_symbolic(dh, N)
        pos_final = T_mats[-1][0:3, 3]

        print("\n2. Calculando Jacobiano Geométrico (Jg)...")
        Jg = calculate_geometric_jacobian(joints, T_mats, N)

        print("\n3. Calculando Jacobiano Analítico (Ja - Euler ZYX)...")
        Ja = calculate_analytical_jacobian(T_mats[-1], q_vars)

        print("\n--- Resultados Simbólicos ---")
        print("Posição Final:")
        sp.pprint(pos_final)

        print("\nJacobiano Geométrico (Simbolico):")
        sp.pprint(Jg)

        print("\nJacobiano Analítico (Simbolico):")
        if N > 3:
            print("[Oculto devido à complexidade. Será exibido na etapa numérica]")
        else:
            sp.pprint(Ja)

        interacao_numerica(Jg, Ja, pos_final, q_vars)

    except KeyboardInterrupt:
        print("\nInterrompido pelo usuário.")
    except Exception as e:
        import traceback
        traceback.print_exc()
        print(f"\nErro fatal: {e}")

if __name__ == '__main__':
    main()